In [3]:
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# AON TEAM: CBU Coding Challenge - OOF Stacking Ensemble
# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=

# ==============================================================================
# 0. ПОДГОТОВКА СРЕДЫ
# ==============================================================================
# Устанавливаем и импортируем все необходимые библиотеки
!pip install catboost -q

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

# Модели
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.linear_model import LogisticRegression

# Предобработка и валидация
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, roc_curve

import warnings
warnings.filterwarnings('ignore')

print("✅ Среда для OOF Stacking готова.")


✅ Среда для OOF Stacking готова.


In [4]:
# ==============================================================================
# 1. ЗАГРУЗКА ДАННЫХ
# ==============================================================================
drive.mount('/content/drive', force_remount=True)
DATA_PATH = '/content/drive/MyDrive/DATA for CBU'
INPUT_FILE = os.path.join(DATA_PATH, 'alldata_engineered.csv')

df = pd.read_csv(INPUT_FILE)
print(f"✅ Обогащенные данные '{os.path.basename(INPUT_FILE)}' загружены. Форма: {df.shape}")

Mounted at /content/drive
✅ Обогащенные данные 'alldata_engineered.csv' загружены. Форма: (89999, 48)


In [5]:
# ==============================================================================
# 2. ОПРЕДЕЛЕНИЕ ПРИЗНАКОВ И ЦЕЛИ
# ==============================================================================
TARGET_COLUMN = 'default'
ID_COLUMNS = ['customer_id', 'application_id']

# Разделяем данные для Submission, если они у вас есть
# Если нет, создадим тестовую выборку для демонстрации
if 'is_test' in df.columns:
    train_df = df[df['is_test'] == 0].copy()
    test_df = df[df['is_test'] == 1].copy()
else: # Демонстрационный сплит
    from sklearn.model_selection import train_test_split
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df[TARGET_COLUMN])

X = train_df.drop(columns=[TARGET_COLUMN] + ID_COLUMNS, errors='ignore')
y = train_df[TARGET_COLUMN]
X_test = test_df.drop(columns=[TARGET_COLUMN] + ID_COLUMNS, errors='ignore')

numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()

# CatBoost требует, чтобы категориальные признаки были строками
for col in categorical_features:
    X[col] = X[col].astype(str)
    X_test[col] = X_test[col].astype(str)

print(f"Train shape: {X.shape}, Test shape: {X_test.shape}")

Train shape: (71999, 45), Test shape: (18000, 45)


In [ ]:
# ==============================================================================
# 3. НАСТРОЙКА OOF-АНСАМБЛЯ
# ==============================================================================
# --- Параметры кросс-валидации ---
N_SPLITS = 5
kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# --- Настройка базовых моделей (гиперпараметры можно найти с Optuna, но эти - хорошая база) ---
lgbm_params = {'objective': 'binary', 'metric': 'auc', 'random_state': 42, 'n_estimators': 2000, 'learning_rate': 0.02, 'num_leaves': 31}
xgb_params = {'objective': 'binary:logistic', 'eval_metric': 'auc', 'random_state': 42, 'n_estimators': 2000, 'learning_rate': 0.02, 'max_depth': 6}
cat_params = {'loss_function': 'Logloss', 'eval_metric': 'AUC', 'random_seed': 42, 'iterations': 2000, 'learning_rate': 0.02, 'depth': 6, 'verbose': 0}

# --- Функция для OOF ---
def run_oof_model(model, X, y, X_test, cat_features_indices):
    oof_preds = np.zeros(X.shape[0])
    test_preds = np.zeros(X_test.shape[0])

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        print(f"--- FOLD {fold+1}/{N_SPLITS} ---")
        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

        # Для CatBoost передаем индексы категорий
        if 'catboost' in str(type(model)):
            model.fit(X_train, y_train, cat_features=cat_features_indices,
                      eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=0)
        else: # Для LGBM и XGBoost
            model.fit(X_train, y_train,
                      eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=0)

        oof_preds[val_idx] = model.predict_proba(X_val)[:, 1]
        test_preds += model.predict_proba(X_test)[:, 1] / N_SPLITS

    return oof_preds, test_preds

In [ ]:
# ==============================================================================
# 4. ЗАПУСК OOF-ПРОЦЕССА
# ==============================================================================
print("\n--- 🚀 Начало OOF-процесса для базовых моделей ---")

# --- Предобработка: CatBoost справится сам, а для LGBM/XGBM нужен OrdinalEncoder ---
preprocessor = ColumnTransformer(transformers=[
    ('num', SimpleImputer(strategy='median'), numerical_features),
    ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_features)
], remainder='passthrough')

X_processed = preprocessor.fit_transform(X)
X_test_processed = preprocessor.transform(X_test)
X_processed_df = pd.DataFrame(X_processed, columns=preprocessor.get_feature_names_out())
X_test_processed_df = pd.DataFrame(X_test_processed, columns=preprocessor.get_feature_names_out())

# --- Запуск моделей ---
# LightGBM
print("\n[1/3] Обучение LightGBM...")
lgbm = lgb.LGBMClassifier(**lgbm_params)
oof_lgbm, test_lgbm = run_oof_model(lgbm, X_processed_df, y, X_test_processed_df, None)

# XGBoost
print("\n[2/3] Обучение XGBoost...")
xgb_model = xgb.XGBClassifier(**xgb_params, use_label_encoder=False)
oof_xgb, test_xgb = run_oof_model(xgb_model, X_processed_df, y, X_test_processed_df, None)

# CatBoost
print("\n[3/3] Обучение CatBoost...")
cat_features_indices = [X.columns.get_loc(col) for col in categorical_features]
catboost = cb.CatBoostClassifier(**cat_params)
oof_cat, test_cat = run_oof_model(catboost, X, y, X_test, cat_features_indices)

# --- Оценка базовых моделей ---
print("\n--- 📈 Оценка OOF-предсказаний базовых моделей ---")
print(f"LGBM OOF AUC: {roc_auc_score(y, oof_lgbm):.5f}")
print(f"XGBM OOF AUC: {roc_auc_score(y, oof_xgb):.5f}")
print(f"CATB OOF AUC: {roc_auc_score(y, oof_cat):.5f}")

In [ ]:
# ==============================================================================
# 5. ОБУЧЕНИЕ МЕТА-МОДЕЛИ (УРОВЕНЬ 1)
# ==============================================================================
print("\n--- 🚀 Обучение мета-модели на OOF-предсказаниях ---")

# Создаем новый тренировочный датасет для мета-модели
meta_X = pd.DataFrame({
    'lgbm_pred': oof_lgbm,
    'xgb_pred': oof_xgb,
    'cat_pred': oof_cat
})

# Создаем новый тестовый датасет для мета-модели
meta_X_test = pd.DataFrame({
    'lgbm_pred': test_lgbm,
    'xgb_pred': test_xgb,
    'cat_pred': test_cat
})

# Обучаем простую, но стабильную мета-модель
meta_model = LogisticRegression(random_state=42)
meta_model.fit(meta_X, y)

# Финальные предсказания
final_predictions = meta_model.predict_proba(meta_X_test)[:, 1]

In [ ]:
# ==============================================================================
# 6. ФИНАЛИЗАЦИЯ И РЕЗУЛЬТАТЫ
# ==============================================================================
print("\n--- 🎉 ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ АНСАМБЛЯ ---")

# Если у вас есть реальные y_test, вы можете посчитать финальный AUC
if TARGET_COLUMN in test_df.columns:
    final_auc = roc_auc_score(test_df[TARGET_COLUMN], final_predictions)
    print(f"Финальный AUC ансамбля на тестовой выборке: {final_auc:.5f}")

# Создание файла для submission
submission_df = pd.DataFrame({
    'application_id': test_df['application_id'], # Убедитесь, что этот столбец есть
    'prediction': final_predictions
})
submission_df.to_csv('submission_oof_stacking.csv', index=False)
print("\n✅ Файл 'submission_oof_stacking.csv' успешно создан!")
print(submission_df.head())